In [69]:
#Parse data
import pandas as pd
import sys
import csv

def download_corpus(infile):

    # read in input file, adjective dictionary
    inp = pd.read_csv(infile)
    #adj_dictionary = pd.read_csv("adj.txt", names=['adj'])['adj'].tolist()
    #dictionary = [x.lower() for x in adj_dictionary if x == x]
    original = pd.read_csv("../data/scraped/marvel-wikia-data-images.csv")
    character_dict = {}
    X_train = []
    y_train = []
    for _, row in original.iterrows():
        character_dict[row["imgurl"]] = row
    for _, row in inp.iterrows():
        if row["Input.image_url"] in character_dict:
            X_item = row[["Answer.emot0", "Answer.emot1", "Answer.emot2", "Answer.pers0", "Answer.pers1", "Answer.pers2", "Answer.phys0", "Answer.phys1", "Answer.phys2"]]
            X_train.append(X_item.tolist())
            y_train.append(character_dict[row["Input.image_url"]]["SEX"])
            #print(character_dict[row["Input.image_url"]])
            
        #print row
    return X_train, y_train
X_train, y_train = download_corpus("../data/results/4_28-sandbox-batch-results.csv")
print(X_train[0:5], y_train[0:5])
corpus = [i for sublist in X_train for i in sublist]
print(corpus[0:5])

([['Anger', 'Stoicism', 'Strength', 'Feirce', 'Tenacious', 'Bold', 'Clawed', 'Strong', 'Invincible'], ['excitement', 'awe', 'inspiration', 'powerful', 'noble', 'strong', 'beared', 'muscular', 'armed'], ['excitement', 'awe', 'happiness', 'powerful', 'bold', 'fierce', 'rock-like', 'large', 'orange'], ['angry', 'upset', 'displeased', 'aggressive', 'violent', 'assertive', 'buff', 'stony', 'strong'], ['thoughtful', 'knowing', 'old', 'smart', 'wise', 'stretchy', 'creepy', 'old', 'bearded']], ['Male Characters', 'Male Characters', 'Male Characters', 'Male Characters', 'Male Characters'])
['Anger', 'Stoicism', 'Strength', 'Feirce', 'Tenacious']


In [62]:
from sklearn.feature_extraction.text import CountVectorizer

In [70]:
X_train_sentence = []
for item in X_train:
    X_row = ""
    for i in item:
        X_row += " " + i + " "
    X_train_sentence.append(X_row)
vectorizer = CountVectorizer()
model = vectorizer.fit(X_train_sentence)

In [71]:
#print(X_train_sentence[0])
X_train_converted = model.transform(X_train_sentence)


In [72]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

clf.fit(X_train_converted, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [73]:
y_pred = clf.predict(X_train_converted)
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.9609756097560975

In [74]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train_converted, y_train, cv=3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.05)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [75]:
from numpy import argmax
features = vectorizer.get_feature_names()
categories = {}
list_of_word_counts = {}
for i in range(0,len(X_train)):
    category = y_train[i]
    words = X_train_converted.toarray()[i]
    if category in categories:
        current = categories[category].copy()
        for c in range(0, len(words)):
            current[c] += words[c]
        categories[category] = current
    else:
        categories[category] = words
for key in categories.keys():
    l = categories[key]
    words = []
    for i in range(10):
        highest = argmax(l)
        words.append((l[highest], features[highest]))
        l[highest] = -1
    print(key)
    print(words)

Genderfluid Characters
[(1, u'brave'), (1, u'green'), (1, u'inspired'), (1, u'manly'), (1, u'powerful'), (1, u'scared'), (1, u'tough'), (1, u'viking'), (1, u'worried'), (0, u'abrasive')]
Male Characters
[(55, u'strong'), (50, u'angry'), (38, u'fear'), (30, u'powerful'), (28, u'evil'), (27, u'mean'), (22, u'determined'), (21, u'excitement'), (21, u'muscular'), (16, u'awe')]
Female Characters
[(19, u'strong'), (13, u'angry'), (12, u'fear'), (11, u'happy'), (10, u'powerful'), (10, u'pretty'), (8, u'cool'), (8, u'evil'), (8, u'red'), (7, u'awe')]
